## First, let's  test the ping method (GET /ping)
This method will be used by Sagemaker for health check our model. It must return a code **200**.

In [ ]:
import json
from urllib import request

base_url='http://localhost:8080'

In [ ]:
%%time

resp = request.urlopen("%s/ping" % base_url)
print("Response code: %d" % resp.getcode() )

## Then we can the predictions (POST /invocations)
This method will be used by Sagemaker for the predictions. Here we're simulating the header parameter related to the CustomAttributes

In [ ]:
import pandas as pd
# Load sample data
df_test = pd.read_csv('input/data/test/test.csv')
df_test.head()

In [ ]:
# Print the unique list of classes
labels = df_test.groupby('class')['amount'].agg(['min','mean','max'])
# Tidy up the index
labels.index = labels.index.str.replace('__label__', '').str.replace('_', ' ')
labels

In [ ]:
%%time
payload = df_test.to_csv(index=False).encode('utf-8')
headers = {"Content-type": "text/csv"}
req = request.Request("%s/invocations" % base_url, data=payload, headers=headers)
resp = request.urlopen(req)
print("Response code: %d: %s" % (resp.getcode(), resp.getheader('Content-Type')))

In [ ]:
# Print the predictions
from io import StringIO
response = resp.read().decode('utf-8')
df_pred = pd.read_csv(StringIO(response))
df_pred[['class_predictions','class_probability']].head()

Verify that we can all pass JSON payload and get back CSV

In [ ]:
%%time
payload = df_test.to_json().encode('utf-8')
headers = {"Content-type": "application/json"}
req = request.Request("%s/invocations" % base_url, data=payload, headers=headers)
resp = request.urlopen(req)
print("Response code: %d: %s" % (resp.getcode(), resp.getheader('Content-Type')))

### Validate Results

Lets look at the precision metric

In [ ]:
from sklearn.metrics import precision_score
avg_prec_micro = precision_score(df_test['class'], df_pred['class_predictions'], average='micro')
avg_prec_macro = precision_score(df_test['class'], df_pred['class_predictions'], average='macro')
print('avg_precision_micro: {}, avg_precision_macro: {}'.format(avg_prec_micro, avg_prec_macro))

Inspect the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, label_names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(label_names))
    plt.xticks(tick_marks, label_names, rotation=45)
    plt.yticks(tick_marks, label_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
cm = confusion_matrix(df_test['class'], df_pred['class_predictions'])
plt.figure(figsize=(15,10))
plot_confusion_matrix(cm, labels.index.values)
plt.show()

print(cm)

## Now, you can go back to the previous Jupyter, stop it and continue executing it